In [1]:
import sys
sys.path.append("../../assets")
# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pickle
import numpy as np
np.set_printoptions(suppress=True)

import bayesflow as beef

/Users/lukas/opt/miniconda3/envs/beef/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Constants

In [ ]:
TRAIN_NETWORKS = True


In [ ]:
# read simulated data for offline training
with open('data/sim_data_offline_learning.pkl', 'rb') as f:
    sim_data = pickle.load(f)

# Training

# Validation

# Evaluation